In [ ]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test= pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
sample_submission=pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
#train= train.sample(frac = 0.01).reset_index(drop = True)
#test= test.sample(frac = 0.01).reset_index(drop = True)

In [ ]:
train.head()

In [ ]:
PRE_TRAINED_MODEL_NAME='bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME,trucation=True, do_lower_case=True)

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.reviews)
    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
         )
        return {
          'review_text': review,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
        'targets': torch.tensor(target, dtype=torch.long)
          }

In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    def forward(self, input_ids, attention_mask):
        returned = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask
          )
        pooled_output = returned["pooler_output"]
        output = self.drop(pooled_output)
        return self.out(output)

In [ ]:
df_train, df_test = train_test_split(
  train,
  test_size=0.1,
  random_state=RANDOM_SEED
)

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = TrainDataset(
    reviews=train.text.to_numpy(),
    targets=train.target.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
     )
    return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
    )
MAX_LEN=64
BATCH_SIZE = 16
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)


In [ ]:
model=SentimentClassifier(2).to(device)
EPOCHS = 10
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
best_loss = np.inf
for epoch in range(10):
    model.train()
    for d in train_data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        outputs = model(
          input_ids=input_ids,
          attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)
        
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
    model = model.eval()
    valid_loss = 0
    with torch.no_grad():
        for d in val_data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
             )
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)
            
            valid_loss += loss.item()
            valid_loss /= len(val_data_loader)
    print(f"EPOCH:{epoch}, Loss:{valid_loss}")
    if valid_loss < best_loss:
        best_loss = valid_loss
        torch.save(model.state_dict(), "nlpmodel.pth")
        print("saved...")    
    

In [ ]:
class TestDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_len):
        self.reviews = reviews
        
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.reviews)
    def __getitem__(self, item):
        review = str(self.reviews[item])
        
        encoding = self.tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
         )
        return {
          'review_text': review,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten()
        
          }

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = TestDataset(
    reviews=test.text.to_numpy(),
    
    tokenizer=tokenizer,
    max_len=max_len
     )
    return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
    )
MAX_LEN=64
BATCH_SIZE = 16
test_data_loader = create_data_loader(test, tokenizer, MAX_LEN, BATCH_SIZE)


In [ ]:
model = model.to(device)
model.load_state_dict(torch.load("./nlpmodel.pth"))

In [ ]:
submit_preds = []

model.eval()
with torch.no_grad():
    for d in test_data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
            
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
             )
        _, preds = torch.max(outputs, dim=1)
        submit_preds.extend(preds)
    predictions = torch.stack(submit_preds).cpu()
len(predictions)

In [ ]:
pred=predictions.cpu().detach().numpy()
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_submission["target"] = pred
print(sample_submission.head())
sample_submission.to_csv("submission.csv", index=False)